In [1]:
import numpy as np
import pandas as pd
from google.cloud import bigquery

In [2]:
client = bigquery.Client.from_service_account_json('automatedreportteams-426703e845d0.json')

In [3]:
previousDay = """
SELECT  date,
        SUM(Spending) as TotalSpending,
        SUM(Clicks) as TotalClicks,
        SUM(Conversions) as TotalConversions,
        ROUND(SUM(Spending)/SUM(Clicks),2) as CostPerClick,
        ROUND(SUM(Spending)/SUM(Conversions),2) as CostPerConversion
FROM `automatedreportteams.transactionData.transactionData` 
WHERE Date = DATE_SUB(CURRENT_DATE(), INTERVAL 1 DAY)
GROUP BY Date
;
"""

In [4]:
def sqlToDataframe(sqlQuery):
    client = bigquery.Client.from_service_account_json('automatedreportteams-426703e845d0.json')
    queryJob = client.query(sqlQuery)
    result = queryJob.result()
    df = pd.DataFrame([dict(i) for i in result])
    return df

In [5]:
previousDayReport = sqlToDataframe(previousDay)

In [6]:
previousDayReport

,date,TotalSpending,TotalClicks,TotalConversions,CostPerClick,CostPerConversion
0,2024-07-31,1798,439,46,4.1,39.09


In [10]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import requests

In [12]:
# Define the scope
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
    
# Add credentials to the account
creds = ServiceAccountCredentials.from_json_keyfile_name('automatedreportteams-426703e845d0.json', scope)
    
# Authorize the clientsheet
client = gspread.authorize(creds)
    
# Open the spreadsheet
spreadsheet = client.open('AutomatedReportTeams')

#Select first sheet
sheet = spreadsheet.sheet1

In [14]:
values = previousDayReport.values.tolist()

In [16]:
values

[[datetime.date(2024, 7, 31), 1798, 439, 46, 4.1, 39.09]]

In [19]:
for row in values:
    row[0] = row[0].strftime('%Y-%m-%d')

In [28]:
sheet.insert_rows(values, 2)

{'spreadsheetId': '1iAGfUqeBVcwxMu5LlxhuoqxEgCvWje7BkQRLfEXE0Wg',
 'updates': {'spreadsheetId': '1iAGfUqeBVcwxMu5LlxhuoqxEgCvWje7BkQRLfEXE0Wg',
  'updatedRange': 'Sheet1!A2:F2',
  'updatedRows': 1,
  'updatedColumns': 6,
  'updatedCells': 6}}

In [27]:
sheet.delete_rows(2)

{'spreadsheetId': '1iAGfUqeBVcwxMu5LlxhuoqxEgCvWje7BkQRLfEXE0Wg',
 'replies': [{}]}